In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
#Acceder a Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Cargar los datos desde el archivo CSV
data= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dataset Multivariable/Mobile phone price.csv")#, skiprows=9, header=None)

In [4]:
# Para eliminar el simbolo '$' de algunos precios de los celulares
columnas_eliminar = ['Price ($)']
simbolo = '$'

for columna in columnas_eliminar:
    data[columna] = data[columna].str.replace(simbolo, '', regex=False)

In [5]:
# Supongamos que 'data' es tu DataFrame
columna_categorica = 'Price ($)'
columna_numerica = 'Price'

# Crea una nueva columna numérica y copia los valores de la columna categórica
data[columna_numerica] = pd.to_numeric(data[columna_categorica], errors='coerce')

# Elimina la columna categórica original
data.drop(columna_categorica, axis=1, inplace=True)

In [6]:
#Convertir variables categoricas a variables numericas
columnas_categoricas = data.select_dtypes(include=['object']).columns
#Convertir variables fechas a variables numericas (si es necesario)
columnas_fechas = data.select_dtypes(include=['datetime64']).columns

In [7]:
#Procesamiento de frases o palabras
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

In [8]:
#Fechas a numeros
for columna in columnas_fechas:
  data[columna] = pd.to_numeric(data[columna].map(datetime.timestamp))

In [9]:
#llenar datos vacios
columnas_especificas = ['Price']
media_columnas = data[columnas_especificas].mean()
data[columnas_especificas] = data[columnas_especificas].fillna(media_columnas)

In [13]:
print(data.columns)

Index(['Brand', 'Model', 'Storage ', 'RAM ', 'Screen Size (inches)',
       'Camera (MP)', 'Battery Capacity (mAh)', 'Price'],
      dtype='object')


In [14]:
# Seleccionar las columnas relevantes para la regresión
features = ['Brand', 'Model', 'Storage ', 'RAM ', 'Screen Size (inches)', 'Camera (MP)','Battery Capacity (mAh)']
target = 'Price'

In [15]:
# Dividir los datos en características y objetivo
X = data[features].values
y = data[target].values

# Escalar las características para facilitar el entrenamiento
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir los datos a tensores de PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [16]:
# Definir el modelo del perceptrón
class Perceptron(nn.Module):
    def __init__(self, input_dim):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        x = self.fc(x)
        return x

# Crear una instancia del modelo
input_dim = X_train.shape[1]
model = Perceptron(input_dim)

In [17]:
# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [18]:
# Entrenar el modelo
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.unsqueeze(1))
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 100/1000, Loss: 47812.671875
Epoch 200/1000, Loss: 43116.06640625
Epoch 300/1000, Loss: 42893.79296875
Epoch 400/1000, Loss: 42865.79296875
Epoch 500/1000, Loss: 42860.12109375
Epoch 600/1000, Loss: 42858.7890625
Epoch 700/1000, Loss: 42858.45703125
Epoch 800/1000, Loss: 42858.3671875
Epoch 900/1000, Loss: 42858.34765625
Epoch 1000/1000, Loss: 42858.33984375


In [19]:
# Evaluar el modelo en el conjunto de prueba
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    test_loss = criterion(outputs, y_test.unsqueeze(1))
    print(f'Loss en el conjunto de prueba: {test_loss.item()}')

Loss en el conjunto de prueba: 44566.046875
